## TO DO:

1. Modify the parser to pull the entire story text instead of just the parts tagged as `<p>`, so it'll pull Miranda's stuff as well.
2. Run scraper for first 8,000 URLs, since it keeps getting hung up for whatever reason.
3. Put the 8,000 results into a DF and then use Python magic to figure out which URLs need scraped, then get the rest of them.
4. Pester A3 for all of 2020's URLs.

## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd
from IPython.display import clear_output

## Set Options

In [2]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 20) ### Default 20
pd.set_option('display.max_colwidth', 1000) ### Default 50

## Data Read-in

<em>All</em> Chicago URLs

In [3]:
df = pd.read_csv('Chicago Stories no Cross post_Piano + GA 2020_Table - Sheet1.csv')

In [4]:
df.columns

Index(['URL', 'Tags'], dtype='object')

In [5]:
len(df)

2023

## Test Ground

In [6]:
# page = requests.get('https://therealdeal.com/chicago/2022/11/01/developer-sells-9m-land-for-super-pricey-planned-lincoln-park-mansion/')
# soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
# soup.find('h1').text.strip() ### Title
# soup.find('h2').text.strip() ### Subhead
# soup.find('div', class_='col-7 col-md-3 order-md-3 text-right').text.strip() ### Pub_Date
# soup.find_all('p')[2].text.strip() ### Lede

In [8]:
# soup.find_all('p')[1].text.strip() ### Lede

In [9]:
# story_text = soup
# story_text_clean = []
# for p in story_text:
#     p = p.text.strip()
#     story_text_clean.append(p)
# story_text_clean = [''.join(story_text_clean)]
# story_text_clean

In [10]:
# story_headlines = soup.find_all('h3')
# story_headlines_clean = []
# for h3 in story_headlines:
#     h3 = h3.text.strip()
#     story_headlines_clean.append(h3)
# story_headlines_clean = [''.join(story_headlines_clean)]

In [11]:
# story_text = soup.find_all('p')[2:]
# story_text_clean = []
# for p in story_text:
#     p = p.text.strip()
#     story_text_clean.append(p)
# story_text_clean = [''.join(story_text_clean)]

TO DO: Fix image scraper to get PNGs...or succesfull append something

In [12]:
# images = soup.find_all('img')
# for image in images:
#     if 'jpg' in image['src']:
#         print(image['src'])

In [13]:
# images = soup.find_all('img')
# png_list = []
# for image in images:
#     if 'png' in image['src']:
#         png_list.append(image['src'])
# print(png_list[-1])

## Scraper

In [14]:
### Create Lists

url_list = []
hed_list = []
dek_list = []
pub_date_list = []
lede_list = []
story_text_list = []
story_headline_list = []

In [15]:
counter = 1
for url in df['URL']:
    if counter > 3000:
        break
    else:
        try:
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            counter = counter + 1
            clear_output(wait=True)
            url_list.append(url)
            print(counter)
            print(url)
            
            ####################
            # HED #
            
            try:
                hed_1 = soup.find('h1').text.strip()
                hed_list.append(hed_1)
            except Exception as e:
                print(f'HED FAIL {url} ERROR: {e}')
                hed_list.append(f'HED FAIL {url} ERROR: {e}')
            
            ####################
            # DEK #
            
            try:
                dek_1 = soup.find('h2').text.strip()
                dek_list.append(dek_1)
            except Exception as e:
                print(f'DEK FAIL {url} ERROR: {e}')
                dek_list.append(f'DEK FAIL {url} ERROR: {e}')
                
            ####################  
            # PUB_DATE #
            
            try:
                pub_date_1 = soup.find('div', class_='col-7 col-md-3 order-md-3 text-right').text.strip()
                pub_date_list.append(pub_date_1)
            except Exception as e:
                print(f'PUB_DATE fail {url} ERROR: {e}')
                pub_date_list.append(f'PUB_DATE FAIL {url} ERROR: {e}')
            
            ####################
            # LEDE #
            
            try:  
                lede_1 = soup.find_all('p')[2].text.strip()
                lede_list.append(lede_1)
            except Exception as e:
                print(f'LEDE fail {url} ERROR: {e}')
                lede_list.append(f'LEDE FAIL {url} ERROR: {e}')
                
            ####################
            # STORY TEXT & H3 headlines #
                
            try:
                story_text = soup.find_all('p')[1:]
                story_text_clean = []
                for p in story_text:
                    p = p.text.strip()
                    story_text_clean.append(p)
                story_text_clean = [''.join(story_text_clean)]
                story_text_list.append(story_text_clean)
                
            except Exception as e:
                print(f'STORY TEXT FAIL {url} ERROR: {e}')
                story_text_list.append(f'STORY TEXT FAIL {url} ERROR: {e}')
                
            try:
                story_headlines = soup.find_all('h3')
                story_headlines_clean = []
                for h3 in story_headlines:
                    h3 = h3.text.strip()
                    story_headlines_clean.append(h3)
                story_headlines_clean = [''.join(story_headlines_clean)]
                story_headline_list.append(story_headlines_clean)
                
            except Exception as e:
                print(f'STORY H3 FAIL {url} ERROR: {e}')
                story_headline_list.append(f'STORY H3 FAIL {url} ERROR: {e}')

        except Exception as e:
            print(f'{url} - FAIL: {e}')
            url_list.append(f'{url} - FAIL: {e}')
            hed_list.append(f'{url} - FAIL: {e}')
            dek_list.append(f'{url} - FAIL: {e}')
            pub_date_list.append(f'{url} - FAIL: {e}')
            lede_list.append(f'{url} - FAIL: {e}')
            story_text_list.append(f'{url} - FAIL: {e}')
            story_headline_list.append(f'{url} - FAIL: {e}')

2024
https://therealdeal.com/chicago/2018/03/27/investor-group-takes-over-landmark-delaware-building/


In [16]:
print(len(url_list))
print('---------')
print(len(hed_list))
print('---------')
print(len(dek_list))
print('---------')
print(len(pub_date_list))
print('---------')
print(len(lede_list))
print('---------')
print(len(story_text_list))
print('---------')
print(len(story_headline_list))
print('---------')

2023
---------
2023
---------
2023
---------
2023
---------
2023
---------
2023
---------
2023
---------


In [17]:
df['URL_2'] = url_list
df['HED'] = hed_list
df['DEK'] = dek_list
df['PUB_DATE'] = pub_date_list
df['LEDE'] = lede_list
df['STORY_TEXT'] = story_text_list
df['STORY_H3_HEADLINES'] = story_headline_list

## Parser Playground

In [18]:
df['STORY_TEXT'] = df['STORY_TEXT'].astype(str)

In [19]:
df['ADDRESS'] = df['STORY_TEXT'].str.extract(r'(?P<ADDRESS>[1-9][0-9]* (?:\w+\W+){1,6}(?:Avenue|Ave|Boulevard|Blvd|Broadway|Court|Deerpath|Drive|Dr|Highway|Huron|Place|Road|Rd|Street|St))')
df['ADDRESS'].isna().value_counts()

False    1409
True      614
Name: ADDRESS, dtype: int64

In [20]:
# df[['URL','ADDRESS']]

In [21]:
df = df.dropna(subset='ADDRESS')
df['ADDRESS'].isna().value_counts()

False    1409
Name: ADDRESS, dtype: int64

In [22]:
# df.to_csv('Chicago_sample.csv')

## Link Formatter

In [23]:
df["short_description"] = df["LEDE"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['LEDE'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["URL"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
# df['description_link']

In [24]:
df['PUB_YEAR'] = df['PUB_DATE'].str.extract(r'(\d{4})')

## Geocoder Setup

In [25]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [26]:
df['geo_address'] = df['ADDRESS'] + ' Chicago, IL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [38]:
df

,URL,Tags,URL_2,HED,DEK,...,loc,point,lat,lon,altitude
2,https://therealdeal.com/chicago/2022/12/09/northwest-side-factory-turned-into-65m-after-rebuild-from-destructive-fire/,"meter,chicago,hermosa,industrial market,northwest side",https://therealdeal.com/chicago/2022/12/09/northwest-side-factory-turned-into-65m-after-rebuild-from-destructive-fire/,Northwest Side factory turned into $65M after rebuild from destructive fire,Newly Weds Foods transferred site to Charles Angell in 2002,...,"(4125 W Wrightwood Ave, Chicago, IL 60639, USA, (41.9273659, -87.73049789999999))","(41.9273659, -87.73049789999999, 0.0)",41.927366,-87.730498,0.0
3,https://therealdeal.com/chicago/2022/12/09/herro-looks-set-to-relist-palmolive-building-condo-with-2m-trim/,"meter,gold coast,housing market,luxury real estate,palmolive building,residential real estate",https://therealdeal.com/chicago/2022/12/09/herro-looks-set-to-relist-palmolive-building-condo-with-2m-trim/,Herro looks set to relist Palmolive Building condo with $2M trim,31st floor of historic building on Mag Mile had star turn in Elle Decor,...,"(159 E Walton Pl, Chicago, IL 60611, USA, (41.8997872, -87.6234987))","(41.8997872, -87.6234987, 0.0)",41.899787,-87.623499,0.0
4,https://therealdeal.com/chicago/2022/12/09/arlington-heights-condos-deconverted-for-10m/,"meter,arlington heights,condo deconversions,condo market,multifamily market,suburban chicago",https://therealdeal.com/chicago/2022/12/09/arlington-heights-condos-deconverted-for-10m/,Arlington Heights condos deconverted for $10M,Interra Realty brokered deal representing both sellers and buyer,...,"(Chicago, IL, USA, (41.8781136, -87.6297982))","(41.8781136, -87.6297982, 0.0)",41.878114,-87.629798,0.0
5,https://therealdeal.com/chicago/2022/12/08/t2-starts-teardown-of-mount-prospect-fire-police-building-for-six-story-project/,"meter,mount prospect,multifamily market,suburban chicago,t2",https://therealdeal.com/chicago/2022/12/08/t2-starts-teardown-of-mount-prospect-fire-police-building-for-six-story-project/,"T2 starts teardown of Mount Prospect fire, police building for six-story project",Wheaton-based developer plans 88 units for suburb’s downtown,...,"(112 E Northwest Hwy, Des Plaines, IL 60016, USA, (42.0558293, -87.91948269999999))","(42.0558293, -87.91948269999999, 0.0)",42.055829,-87.919483,0.0
6,https://therealdeal.com/chicago/2022/12/08/glenview-complex-sells-for-discount-in-rare-multifamily-loss/,"meter,fortress investment group,multifamily market,suburban chicago",https://therealdeal.com/chicago/2022/12/08/glenview-complex-sells-for-discount-in-rare-multifamily-loss/,Glenview complex sells for discount in rare multifamily loss,BJB Properties bought the 425-unit complex for $67.2M,...,"(4300 W Lake St, Chicago, IL 60624, USA, (41.88628509999999, -87.7338357))","(41.88628509999999, -87.7338357, 0.0)",41.886285,-87.733836,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2017,https://therealdeal.com/chicago/2018/04/03/blackstone-takes-out-record-1-3b-refi-for-willis-tower/,"archive-paywall,blackstone group,commercial real estate,real estate finance,willis tower",https://therealdeal.com/chicago/2018/04/03/blackstone-takes-out-record-1-3b-refi-for-willis-tower/,Blackstone takes out record $1.3B+ refi for Willis Tower,Firm paid $1.3B for city's tallest skyscraper in 2015,...,"(N Morgan St, Chicago, IL, USA, (41.8889966, -87.652239))","(41.8889966, -87.652239, 0.0)",41.888997,-87.652239,0.0
2018,https://therealdeal.com/chicago/2018/04/02/will-compass-chicago-ambitions-send-rivals-spinning/,"archive-paywall,@properties,compass,residential real estate,tech",https://therealdeal.com/chicago/2018/04/02/will-compass-chicago-ambitions-send-rivals-spinning/,Will Compass’ Chicago ambitions send its rivals spinning?,“We are not some evil empire trying to take over the world.”,...,"(3033 N Clark St, Chicago, IL 60657, USA, (41.9372941, -87.6483622))","(41.9372941, -87.6483622, 0.0)",41.937294,-87.648362,0.0
2020,https://therealdeal.com/chicag

In [39]:
df.columns

Index(['URL', 'Tags', 'URL_2', 'HED', 'DEK', 'PUB_DATE', 'LEDE', 'STORY_TEXT',
       'STORY_H3_HEADLINES', 'ADDRESS', 'short_description', 'remaining_desc',
       'description_link', 'PUB_YEAR', 'geo_address', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [40]:
df2 = df.drop(columns=['STORY_H3_HEADLINES','PUB_DATE','STORY_TEXT','short_description','LEDE','remaining_desc'])
df2 = df2.dropna(subset='loc')
df2.drop(df2.loc[df2['DEK'].str.contains('FAIL')].index, inplace=True)
df2.drop(df2.loc[df2['HED'].str.contains('FAIL')].index, inplace=True)
df2.reset_index(inplace=True)
df2 = df2.drop(columns='index')

In [44]:
from pandas_geojson import to_geojson, write_geojson

In [42]:
df2['URL_2'].str.contains('FAIL').value_counts()

False    1401
Name: URL_2, dtype: int64

In [47]:
df2.PUB_YEAR.value_counts()

2022    715
2019    233
2021    179
2018    163
2020    111
Name: PUB_YEAR, dtype: int64

In [58]:
df_2018 = df2.loc[df2['PUB_YEAR'] == '2018']
df_2019 = df2.loc[df2['PUB_YEAR'] == '2019']
df_2020 = df2.loc[df2['PUB_YEAR'] == '2020']
df_2021 = df2.loc[df2['PUB_YEAR'] == '2021']
df_2022 = df2.loc[df2['PUB_YEAR'] == '2022']

In [63]:
geo_json = to_geojson(df=df_2022, lat='lat', lon='lon',
                     properties=['HED','DEK','description_link','PUB_YEAR','geo_address'])
write_geojson(geo_json, filename='Chicago_2022.geojson', indent=4)

In [64]:
df2.to_csv('Chicago_data_final.csv')

In [34]:
# df[['geo_address','lat','lon','altitude']]

## Code graveyard

In [35]:
            ####################
            # IMAGES #
            
#             try:            
#                 images = soup.find_all('img')
#                 image_found = False
#                 try:
#                     while image_found == False:
#                         png_list = []
#                         for image in images:
#                             if 'jpg' in image['src']:
#                                 image_list.append(image['src'])
#                                 print(image['src'])
#                                 image_found = True
#                             break
#                         if image_found == False:
#                             for image in images:
#                                 if 'png' in image['src']:
#                                     png_list.append(image['src'])
#                                     image_found = True
#                             image_list.append(png_list[-1])
#                             print(png_list[-1])
#                 except Exception as e:
#                     print(f'No dice: {e}')
#                 print('------')
#             except Exception as e:
#                 print('find_image_fail')
            
            ####################

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [37]:
cwd = 'chicago_transaction_scraper_map'

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chicago_transaction_scraper_map
